<a href="https://colab.research.google.com/github/PKRaj007/PKRaj007/blob/main/CVDL_Project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
%cd /content/drive/MyDrive/Computer Vision/smoke_fog_dataset

/content/drive/.shortcut-targets-by-id/1l0l7QH5lS8z8L_MD-p6GX6kgZjzvhSYF/smoke_fog_dataset


In [ ]:
path='/content/drive/MyDrive/Computer Vision/smoke_fog_dataset/test/smoke/'

In [ ]:
!ls

'dataset introduction.docx'   test   train


In [ ]:
import matplotlib.pyplot as plt

#Preprocessing

In [ ]:
img = plt.imread(path +'smoke.994.jpg')

In [ ]:
img.shape

(592, 1280, 3)

Rescaling Image using ImageDataGenerator

In [ ]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [ ]:
train_data = ImageDataGenerator(rescale=1/255)
#val_data = ImageDataGenerator(rescale=1/255)
test_data = ImageDataGenerator(rescale=1/255)

In [ ]:
train_generator = train_data.flow_from_directory('train', target_size=(224,224), batch_size=8, class_mode='categorical')

Found 26980 images belonging to 4 classes.


In [ ]:
test_generator = test_data.flow_from_directory('test', target_size=(224,224), batch_size=8,shuffle = False,class_mode='categorical')

Found 6730 images belonging to 4 classes.


In [ ]:
y_true = test_generator.classes
y_true

array([0, 0, 0, ..., 3, 3, 3], dtype=int32)

# Designing the proposed vgg 16 model with Attention Mechanism and Feature Level and Decision Level Fusion as given in paper

# Designing the Model

In [ ]:
import tensorflow as tf

from tensorflow.keras import models
from tensorflow.keras import layers

In [ ]:
import cv2
import numpy as np
import os
from keras.preprocessing.image import ImageDataGenerator
from keras.layers import Dense,Flatten,Conv2D,Activation,Dropout
from keras import backend as K

import keras
from keras.models import Sequential, Model
from keras.models import load_model
from tensorflow.keras.optimizers import SGD
from keras.callbacks import EarlyStopping,ModelCheckpoint
from keras.layers import MaxPool2D
from keras.layers.pooling import AvgPool2D
from google.colab.patches import cv2_imshow
from tensorflow.keras.layers import *
from keras import Input
from keras.layers.convolutional import UpSampling2D

# from tensorflow.keras import Add,Multiply

In [ ]:
 # Spatial Attention Mechanism

  # avg_pool_layer = AvgPool2D(1,(1,1))(vgg_layer2)
  # max_pool_layer = MaxPool2D(1,(1,1))(vgg_layer2)
  # add_layer = Concatenate()([avg_pool_layer,max_pool_layer])

  # Channel Attention Mechanism

  # global_avg_pool_layer = GlobalAvgPool2D()(spatial_attention_output)
  # global_max_pool_layer = GlobalMaxPool2D()(b)

  #dense_avg_layer = Dense(1)(global_avg_pool_layer)
  #dense_max_layer = Dense(1)(global_max_pool_layer)

  # add_layer = Add()([dense_avg_layer, dense_max_layer])
  # mul_layer = Multiply()([add_layer, b])

In [ ]:
def vggSmokeDetectionModel(input, n/um_classes):
  # Adding VGG Layer 1 and 2
  input = Input(shape = (224,224,3))
  #changing filters from 64 -> 16
  input_layer = Conv2D(filters=16, kernel_size=(3,3), padding="same", activation="relu")(input)
  input_layer = Conv2D(filters=64, kernel_size=(3,3), padding="same", activation="relu")(input_layer)

  max_pool_layer1 = MaxPool2D(pool_size=(2,2), strides=(2,2))(input_layer)
  #changing filters from 128 -> 32/64
  vgg_layer1 = Conv2D(filters=32, kernel_size=(3,3), padding="same", activation="relu")(max_pool_layer1)
  vgg_layer1 = Conv2D(filters=64, kernel_size=(3,3), padding="same", activation="relu")(vgg_layer1)

  max_pool_layer2 = MaxPool2D(pool_size=(2,2), strides=(2,2))(vgg_layer1)
  #changing filters from 256 -> 32/64
  vgg_layer2 = Conv2D(filters=32, kernel_size=(3,3), padding="same", activation="relu")(max_pool_layer2)
  vgg_layer2 = Conv2D(filters=32, kernel_size=(3,3), padding="same", activation="relu")(vgg_layer2)
  vgg_layer2 = Conv2D(filters=64, kernel_size=(3,3), padding="same", activation="relu")(vgg_layer2)
  
  # Adding Spatial Attention
  conv_spatial = Conv2D(filters=64, kernel_size=(3,3), padding='same', activation="relu")(vgg_layer2)
  weight_spatial = Activation("sigmoid")(conv_spatial)
  prod_spatial = Multiply()([vgg_layer2, weight_spatial])
  spatial_attention_output = Add()([vgg_layer2, prod_spatial])

  # Adding Channel Attention
  conv_channel = Conv2D(filters=64, kernel_size=(1,1), padding="same", activation="relu")(spatial_attention_output)
  global_avg_pool_layer = GlobalAvgPool2D()(conv_channel)
  weight_channel = Activation("sigmoid")(conv_channel)
  prod_channel = Multiply()([spatial_attention_output,weight_channel])
  channel_attention_output = Add()([spatial_attention_output, prod_channel])

  # Applying Feature-Level Fusion

  # Adding vgg_layer 3
  vgg_maxPool_layer3 = MaxPool2D(pool_size=(2,2), strides=(2,2))(channel_attention_output)
  #changing filters from 512 to 64/128
  vgg_layer3 = Conv2D(filters=64, kernel_size=(3,3), padding="same", activation="relu")(vgg_maxPool_layer3)
  vgg_layer3 = Conv2D(filters=64, kernel_size=(3,3), padding="same", activation="relu")(vgg_layer3)
  vgg_layer3 = Conv2D(filters=128, kernel_size=(3,3), padding="same", activation="relu")(vgg_layer3)

  # Adding vgg_layer 4
  vgg_maxPool_layer4 = MaxPool2D(pool_size=(2,2), strides=(2,2))(vgg_layer3)
  #changing filters from 512 to 64/128
  vgg_layer4 = Conv2D(filters=64, kernel_size=(3,3), padding="same", activation="relu")(vgg_maxPool_layer4)
  vgg_layer4 = Conv2D(filters=64, kernel_size=(3,3), padding="same", activation="relu")(vgg_layer4)
  vgg_layer4 = Conv2D(filters=128, kernel_size=(3,3), padding="same", activation="relu")(vgg_layer4)

  # Adding vgg_layer 5
  vgg_maxPool_layer5 = MaxPool2D(pool_size=(2,2), strides=(2,2))(vgg_layer4)
  vgg_layer5 = vgg_maxPool_layer5

  # Applying 1*1 Convolution on vgg layer 5
  #changing filters from 512 to 128
  vgg_layer5_Conv = Conv2D(filters=128, kernel_size=(1,1), padding="same", activation="relu")(vgg_layer5)

  # #Upsampling vgg_layer 5
  vgg_layer5_Conv_Upsampled = UpSampling2D(size=(2,2))(vgg_layer5_Conv)

  # Applying 1*1 Convolution on vgg layer 4
  #changing filters from 512 to 128
  vgg_layer4_Conv = Conv2D(filters=128, kernel_size=(1,1), padding="same", activation="relu")(vgg_layer4)

  # Adding vgg_layer4_Conv with vgg_layer5_Conv_Upsampled
  add_vgg_layer4_vgg_layer5 = Add()([vgg_layer4_Conv, vgg_layer5_Conv_Upsampled])

  # UpSampling the Concatenated Result
  add_vgg_layer4_vgg_layer5_upsampled = UpSampling2D(size=(2,2))(add_vgg_layer4_vgg_layer5)

  # Applying 1*1 Convolution on vgg layer 3
  #changing filters from 512 to 128
  vgg_layer3_Conv = Conv2D(filters=128, kernel_size=(1,1), padding="same", activation="relu")(vgg_layer3)

  # adding convolved vgg layer 3 with upsampled added vgg layer 4 and 5
  add_all_layers = Add()([vgg_layer3_Conv, add_vgg_layer4_vgg_layer5_upsampled])

  # Applying 3*3 Convolution Smoothing on vgg_layer 5
  #changing filters from 512 to 128
  vgg_layer5_smoothed = Conv2D(filters=128, kernel_size=(3,3), padding="same", activation="relu")(vgg_layer5)

  # Applying 3*3 Convolution Smoothing on added upsampled convolved vgg layer 5 and convolved vgg layer 4
  #changing filters from 512 to 128
  add_vgg_layer4_vgg_layer5_smoothed = Conv2D(filters=128, kernel_size=(3,3), padding="same", activation="relu")(add_vgg_layer4_vgg_layer5)

  # Applying 3*3 Convolution Smoothing on added all layers
  #changing filters from 512 to 128
  add_all_layer_smoothed = Conv2D(filters=128, kernel_size=(3,3), padding="same", activation="relu")(add_all_layers)

  # Applying Decision Level Fusion

  # Adding Classifier 1
  #changing filters from 512 to 128
  conv_classifier1 = Conv2D(filters=128, kernel_size=(1,1), padding="same", activation="relu")(vgg_layer5_smoothed)
  dropout_classifier1 = Dropout(rate=0.2, seed=None)(conv_classifier1)
  globalMaxPool_classifier1 = GlobalMaxPool2D()(dropout_classifier1)

  dense_classifier1 = Dense(num_classes, activation = 'sigmoid')(globalMaxPool_classifier1)

  # Adding Classifier 2
  #changing filters from 512 to 128
  conv_classifier2 = Conv2D(filters=128, kernel_size=(1,1), padding="same", activation="relu")(add_vgg_layer4_vgg_layer5_smoothed)
  dropout_classifier2 = Dropout(rate=0.2, seed=None)(conv_classifier2)
  globalMaxPool_classifier2 = GlobalMaxPool2D()(dropout_classifier2)

  dense_classifier2 = Dense(num_classes, activation = 'sigmoid')(globalMaxPool_classifier2)

  # Adding Classifier 3
  #changing filters from 512 to 128
  conv_classifier3 = Conv2D(filters=128, kernel_size=(1,1), padding="same", activation="relu")(add_all_layer_smoothed)
  dropout_classifier3 = Dropout(rate=0.2, seed=None)(conv_classifier3)
  globalMaxPool_classifier3 = GlobalMaxPool2D()(dropout_classifier3)

  dense_classifier3 = Dense(num_classes, activation = 'softmax')(globalMaxPool_classifier3)
 
  model = Model(inputs = input, outputs = [dense_classifier1, dense_classifier2, dense_classifier3])
  return model

input = Input(shape = (224,224,3))
vgg_model = vggSmokeDetectionModel(input, 4)
vgg_model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_2 (InputLayer)           [(None, 224, 224, 3  0           []                               
                                )]                                                                
                                                                                                  
 conv2d (Conv2D)                (None, 224, 224, 16  448         ['input_2[0][0]']                
                                )                                                                 
                                                                                                  
 conv2d_1 (Conv2D)              (None, 224, 224, 64  9280        ['conv2d[0][0]']                 
                                )                                                             

In [ ]:
# vgg_model.parameters()

# Training the Model

In [ ]:
from tensorflow.keras.optimizers import SGD
import tensorflow.keras.metrics as tfm

# from decimal import *

In [ ]:
opt = SGD(learning_rate=0.001)

vgg_model.compile(optimizer=opt, 
              loss={
                  'dense': 'categorical_crossentropy', 
                  'dense_1': 'categorical_crossentropy', 
                  'dense_2': 'categorical_crossentropy'},
              loss_weights={
                  'dense': 1, 
                  'dense_1': 1, 
                  'dense_2': 1},
              metrics=['accuracy'])

In [ ]:
vgg_model_results = vgg_model.fit(train_generator, epochs=30)

Epoch 1/30
3373/3373 [==============================] - 7732s 2s/step - loss: 3.0930 - dense_loss: 1.0408 - dense_1_loss: 1.0310 - dense_2_loss: 1.0212 - dense_accuracy: 0.5326 - dense_1_accuracy: 0.5279 - dense_2_accuracy: 0.5348
Epoch 2/30
3373/3373 [==============================] - 195s 58ms/step - loss: 1.5617 - dense_loss: 0.5190 - dense_1_loss: 0.5222 - dense_2_loss: 0.5206 - dense_accuracy: 0.7683 - dense_1_accuracy: 0.7656 - dense_2_accuracy: 0.7646
Epoch 3/30
3373/3373 [==============================] - 193s 57ms/step - loss: 0.9227 - dense_loss: 0.2931 - dense_1_loss: 0.3099 - dense_2_loss: 0.3197 - dense_accuracy: 0.8810 - dense_1_accuracy: 0.8734 - dense_2_accuracy: 0.8696
Epoch 4/30
3373/3373 [==============================] - 193s 57ms/step - loss: 0.5176 - dense_loss: 0.1634 - dense_1_loss: 0.1665 - dense_2_loss: 0.1878 - dense_accuracy: 0.9394 - dense_1_accuracy: 0.9378 - dense_2_accuracy: 0.9326
Epoch 5/30
3373/3373 [==============================] - 192s 57ms/step - 

In [ ]:
vgg_model_results = vgg_model.fit(train_generator, epochs=30)

Epoch 1/30
3373/3373 [==============================] - 4603s 1s/step - loss: 3.5905 - dense_loss: 1.2067 - dense_1_loss: 1.2076 - dense_2_loss: 1.1762 - dense_accuracy: 0.4345 - dense_1_accuracy: 0.4182 - dense_2_accuracy: 0.4552
Epoch 2/30
3373/3373 [==============================] - 384s 114ms/step - loss: 1.6341 - dense_loss: 0.5414 - dense_1_loss: 0.5503 - dense_2_loss: 0.5424 - dense_accuracy: 0.7559 - dense_1_accuracy: 0.7510 - dense_2_accuracy: 0.7520
Epoch 3/30
3373/3373 [==============================] - 385s 114ms/step - loss: 0.8333 - dense_loss: 0.2540 - dense_1_loss: 0.2783 - dense_2_loss: 0.3010 - dense_accuracy: 0.8971 - dense_1_accuracy: 0.8895 - dense_2_accuracy: 0.8780
Epoch 4/30
3373/3373 [==============================] - 383s 114ms/step - loss: 0.4028 - dense_loss: 0.1193 - dense_1_loss: 0.1268 - dense_2_loss: 0.1568 - dense_accuracy: 0.9580 - dense_1_accuracy: 0.9552 - dense_2_accuracy: 0.9437
Epoch 5/30
3373/3373 [==============================] - 383s 114ms/ste

In [ ]:
callbacks = [
             tf.keras.callbacks.ModelCheckpoint(
                 '/content/drive/MyDrive/Computer Vision/smoke_fog_dataset/model_save.h5',
                 save_best_only=True,
                 monitor='accuracy'
             )
]

In [ ]:
vgg_model.fit(train_generator, epochs=10,callbacks=callbacks)

Epoch 1/10
3373/3373 [==============================] - 6046s 2s/step - loss: 2.9190 - dense_loss: 0.9721 - dense_1_loss: 0.9770 - dense_2_loss: 0.9700 - dense_accuracy: 0.5478 - dense_1_accuracy: 0.5394 - dense_2_accuracy: 0.5399
Epoch 2/10
3373/3373 [==============================] - 510s 151ms/step - loss: 0.9773 - dense_loss: 0.3109 - dense_1_loss: 0.3333 - dense_2_loss: 0.3331 - dense_accuracy: 0.8759 - dense_1_accuracy: 0.8648 - dense_2_accuracy: 0.8640
Epoch 3/10
3373/3373 [==============================] - 510s 151ms/step - loss: 0.4357 - dense_loss: 0.1322 - dense_1_loss: 0.1453 - dense_2_loss: 0.1582 - dense_accuracy: 0.9526 - dense_1_accuracy: 0.9472 - dense_2_accuracy: 0.9442
Epoch 4/10
3373/3373 [==============================] - 509s 151ms/step - loss: 0.2427 - dense_loss: 0.0717 - dense_1_loss: 0.0766 - dense_2_loss: 0.0943 - dense_accuracy: 0.9765 - dense_1_accuracy: 0.9760 - dense_2_accuracy: 0.9687
Epoch 5/10
3373/3373 [==============================] - 512s 152ms/ste

In [ ]:
from keras.models import load_model

In [ ]:
model = load_model('/content/drive/MyDrive/Computer Vision/smoke_fog_dataset/model_save.h5')

In [ ]:
model.fit(train_generator, epochs=10,callbacks=callbacks)

In [ ]:
model = load_model('/content/drive/MyDrive/Computer Vision/smoke_fog_dataset/model_save.h5')

In [ ]:
model.fit(train_generator, epochs=10,callbacks=callbacks)

In [ ]:
train_accuracy = vgg_model_results.history['accuracy']
print(train_accuracy)

train_loss = vgg_model_results.history['loss']
print(train_loss)

# Testing the model

In [ ]:
vgg_model_test_loss, vgg_model_test_acc = vgg_model.evaluate(test_generator)

# Plotting Confusion Matrix

In [ ]:
from sklearn.metrics import confusion_matrix
import matplotlib.pyplot as plt
import numpy as np
import itertools  

In [ ]:
y_true = test_generator.classes
y_pred = vgg_model.predict(test_generator)

In [ ]:
target_names = []
for key in train_generator.class_indices:
  target_names.append(key)

print(target_names)

['non_smokefog', 'nonsmoke', 'smoke', 'smoke_fog']


In [ ]:
def plot_confusion_matrix(cm, classes, normalize=True, title='Confusion matrix', cmap=plt.cm.Blues):
    plt.figure(figsize=(10,10))
    plt.imshow(cm, interpolation='nearest', cmap=cmap)
    plt.title(title)
    plt.colorbar()
    tick_marks = np.arange(len(classes))
    plt.xticks(tick_marks, classes, rotation=45)
    plt.yticks(tick_marks, classes)
    
    if normalize:
        cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]
        cm = np.around(cm, decimals=2)
        cm[np.isnan(cm)] = 0.0
        print("Normalized confusion matrix")

    else:
        print('Confusion matrix, without normalization')
    thresh = cm.max() / 2.

    for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
        plt.text(j, i, cm[i, j],
                 horizontalalignment="center",
                 color="white" if cm[i, j] > thresh else "black")
    plt.tight_layout()
    plt.ylabel('True label')
    plt.xlabel('Predicted label')

In [ ]:
cm = confusion_matrix(y_true, y_pred)
plot_confusion_matrix(cm, target_names, title='Confusion Matrix')

# Estimating the model based on Quantative Evaluations

In [ ]:
from sklearn.metrics import classification_report

In [ ]:
y_true = test_generator.classes
y_pred = vgg_model.predict(test_generator)

In [ ]:
print(classification_report(y_true, y_pred, target_names=target_names))